# Kappa Architecture hands-on lab

During the previous courses, you've been introduced to a number of tools and techniques to store and process data. This hands-on lab explains how you can combine these tools and techniques to get an end-to-end data processing pipeline.

## Use case: a Flemish website

Throughout this lab, you will build a platform to capture and analyze traffic to a website. You will build the following components:

* An HTTP endpoint to gather pageview data from a website
* A data cleaning pipeline for the pageview data
* A security alerting system that detects DDoS attacks and requests trying to access your management platform.
* A real-time dashboard showing the number of users on your website at any time.

You'll also learn how to load the pageview data into a time-series database so it's accessible for Business Intelligence tools.

The dataset used in this lab is a three-hour window of page view data of a popular Flemish news website. The dataset is anonymized and the URL's are changed to point to Wikipedia pages in order to protect the privacy of the users and the website owner. Although the dataset of this three hour window is only 500 MB in size, the original dataset's compressed size is ~1 TB. We used a similar setup to the one explained in the lab to analyze and predict the popularity of news articles.

## Introduction to Kappa Architecture

[The Kappa Architecture](http://milinda.pathirage.org/kappa-architecture.com/) is a Big Data processing design pattern. It is a high-level description of how to combine data analytics tools to solve real problems. This isn't a one size fits all solution. Many other data processing design patterns exist such as [the Lambda architecture](https://databricks.com/glossary/lambda-architecture) and [the Zeta architecture](https://www.waitingforcode.com/general-big-data/zeta-architecture/read).


The Kappa architecture focuses around "events" and uses an event log as the source of truth. So instead of storing the final state of your data you store each individual event, so for the platform of this lab we'll store each individual page load and use that event log as our source of truth. We then calculate the visitors per minute from that event log and store the result in a *serving database*. Apps and tools who need to know the number of visitors connect to the serving database to get their information, just like they would in a traditional data processing architecture. The actual raw page view data, however, is still available for reprocessing. The Kappa architecture is ideal for time-series data such as page views, logs or IoT sensors. 


Below you see the Kappa architecture applied to our website pageviews example.

1. The website contains javascript code that sends an HTTP POST request to the ingest API when the user first loads the website and when the user scrolls down.
1. The ingest API publishes each individual event on the "clicks" topic on Kafka, a distributed queue.
1. Spark Streaming jobs subscribe to the "clicks" topic, process the clicks and write the processed data to serving databases and new Kafka topics.
1. Applications such as the Tableau BI platform or Grafana connect to a serving database and use the data.

<img src="img/Big Data Hands On - Kappa.png">

This has enormous flexibility.

* If you update the stream processing code that generates the databases, you can easily replay all the events in order to create a new version of the serving layer. This can happen in parallel to the existing serving layer.
* It's easy to have multiple different types of databases in the serving layer, each optimized for a specific application but built from the same dataset.
* It's easily horizontally scalable but can still integrate with non-scalable applications.





## Note 1

Since these technologies are meant to run on servers processing large amounts of data, they are quite resource intensive. For this reason, we do not run the traffic simulator live. Instead, the traffic simulator will run a single time, send a number of requests in batch, and exit.

As a result of this, during development, your streaming queries will not process _live_ data. Instead, they will process the historical data on a Kafka topic, exit, and show you the result. This makes sure that as few components as possible need to run at the same time. This also ensures the result of your query always uses the exact same data, so you can see your changes to the query clearly and compare them to previous versions of your query.

## Note 2

**Always save** the notebook **before** running cells. If there is a resource problem your IDE might crash and you would lose your work.

# Exercises

Below is the architecture of the Kappa-inspired platform you'll build in this lab. Specifically, you'll create the following components, each in their own notebook. Start with the first component and come back to this notebook to continue with the next component when the first component is finished.

1. Create an ingest API using the Python Flask framework in `1a-endpoint.ipynb`. Use `1b-fake-website.ipynb` to send fake clicks to your endpoint.
1. Clean the data using Spark Structured Streaming in `2-cleanup.ipynb`. This will use the data from the ingest API.
1. Generate security alerts in `3c-security.ipynb` and generate a notification when such an alert happens in `3dsecurity-notifications.ipynb)
1. Create a realtime dashboard using Spark in `4a-dashboard-generation.ipynb` and show the dashboard in `4b-dashboard-visualization.ipynb`.

<img src="img/use-case-overview.png">
